In [1]:
import pandas as pd
import requests
import json
import glob, os
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
import certifi
api = 'c4ad87b03bec1878bc0be6156d4472b0'
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 100)

In [2]:
prices_request = requests.get(f'https://financialmodelingprep.com/api/v3/available-traded/list?apikey={api}').json()




In [3]:
df_ticker_list = pd.json_normalize(prices_request)


In [4]:
df_ticker_list

,symbol,name,price,exchange,exchangeShortName
0,5JS.SI,Indofood Agri Resources Ltd.,0.31,SES,EURONEXT
1,JMUEX,JPMorgan U.S. Equity Fund,17.06,Nasdaq,NASDAQ
2,CSRYX,Columbia Select Large Cap Value Fund,31.56,Nasdaq,NASDAQ
3,NWXQX,Nationwide Mid Cap Market Index Fund,16.02,Nasdaq,NASDAQ
4,GRSO,"Grow Solutions Holdings, Inc.",0.00,Other OTC,PNK
5,4270.SR,Saudi Printing and Packaging Co.,16.58,Saudi,SAU
6,4828.T,Business Engineering Corporation,"2,357.00",Tokyo,JPX
7,MDY,SPDR S&P MIDCAP 400 ETF Trust,437.19,New York Stock Exchange Arca,AMEX
8,PWZ,Invesco California AMT-Free Municipal Bond ETF,24.21,New York Stock Exchange Arca,AMEX
9,SGAS.DE,iShares MSCI USA ESG Screened UCITS ETF,6.89,XETRA,XETRA


In [5]:
df_ticker_list['exchange'].value_counts()
list_of_us_exchanges = ['New York Stock Exchange','NASDAQ Capital Market','NASDAQ Global Select',
                       'NASDAQ Global Market']
df_nasdaq = df_ticker_list[df_ticker_list['exchange'].isin(list_of_us_exchanges)]

In [6]:
df_nasdaq

,symbol,name,price,exchange,exchangeShortName
10,USLB,Invesco Russell 1000 Low Beta Equal Weight ETF,37.26,NASDAQ Global Market,NASDAQ
11,BLUE,"bluebird bio, Inc.",7.42,NASDAQ Global Select,NASDAQ
13,RFACR,RF Acquisition Corp.,0.07,NASDAQ Global Market,NASDAQ
14,LNN,Lindsay Corporation,165.84,New York Stock Exchange,NYSE
17,WAL-PA,Western Alliance Bancorporation,20.95,New York Stock Exchange,NYSE
18,OFSSH,OFS Capital Corporation 4.95% Notes due 2028,21.20,NASDAQ Global Select,NASDAQ
19,NTLA,"Intellia Therapeutics, Inc.",36.03,NASDAQ Global Market,NASDAQ
33,BLND,"Blend Labs, Inc.",1.22,New York Stock Exchange,NYSE
42,FRT-PC,Federal Realty Investment Trust,21.05,New York Stock Exchange,NYSE
43,FYT,First Trust Small Cap Value AlphaDEX Fund,44.69,NASDAQ Global Market,NASDAQ


In [7]:
df_nasdaq_sort = df_nasdaq[df_nasdaq['price']>9.99]

In [9]:
df_nasdaq_tickers = pd.read_excel('NASDAQ_tickers.xlsx')

In [15]:
df_tech_tickers = pd.read_excel('NASDAQ_tickers.xlsx', sheet_name='GICS - Industry')

In [17]:
df_tech_tickers.head()

,Company,Ticker,Industry
0,KBR INC,KBR,IT Services
1,GTT COMMUNICATIONS INC,GTT,IT Services
2,INPIXON,INPX,IT Services
3,EURONET WORLDWIDE INC,EEFT,IT Services
4,INTERNATIONAL MONEY EXPR INC,IMXI,IT Services


In [18]:
df_tech_sector_tickers = pd.read_excel('NASDAQ_tickers.xlsx', sheet_name='GICS - Sector')

In [20]:
df_tech_sector_tickers.head()

,Company,Ticker,Industry,Sector
0,KBR INC,KBR,IT Services,IT Consulting & Services
1,INPIXON,INPX,IT Services,IT Consulting & Services
2,AMERI HOLDINGS INC,AMRH,IT Services,IT Consulting & Services
3,BOOZ ALLEN HAMILTON HLDG CP,BAH,IT Services,IT Consulting & Services
4,COGNIZANT TECH SOLUTIONS,CTSH,IT Services,IT Consulting & Services


In [30]:
def get_mcap(ticker):
    
    
    prices_request = requests.get(f'https://financialmodelingprep.com/api/v3/historical-market-capitalization/{ticker}?limit=1260&apikey={api}').json()
    

    #dates_df = pd.DataFrame.from_dict(prices_request,orient='columns')
    out_file = open(f"MarketCap/{ticker}.json", "w") 
    json.dump(prices_request, out_file, indent = 4) 
    out_file.close() 
    
    with open(f'MarketCap/{ticker}.json','r') as f:
        data = json.loads(f.read())
    # Flatten data
    df_nested_list = pd.json_normalize(data)
    df_nested_list['Ticker'] = ticker
    df_nested_list.to_csv(f"MarketCap/{ticker}.csv")
    #return df_nested_list


In [31]:
tickers = df_tech_sector_tickers['Ticker']

In [32]:
# call the function
result = [get_mcap(ticker) for ticker in tickers]

In [34]:
path = '/Users/brendan/Desktop/Python/Software 2020/MarketCap'
csv_files = [f for f in os.listdir(path) if f.endswith('csv')]
raw_data_df = []
for file in csv_files:
    file = pd.read_csv('/Users/brendan/Desktop/Python/Software 2020/MarketCap/'+file)
    df = pd.DataFrame(file)
    #df['ticker'] = file
    raw_data_df.append(df)
master_df = pd.concat(raw_data_df, ignore_index=True)
master_df = master_df.drop('Unnamed: 0',axis=1)
master_df

,symbol,date,marketCap,Ticker,Error Message
0,CSCO,2022-12-20,"195,526,279,168.00",CSCO,NaN
1,CSCO,2022-12-19,"194,724,086,900.00",CSCO,NaN
2,CSCO,2022-12-16,"196,408,409,200.00",CSCO,NaN
3,CSCO,2022-12-15,"197,805,164,300.00",CSCO,NaN
4,CSCO,2022-12-14,"202,529,482,800.00",CSCO,NaN
5,HUBS,2022-12-20,"12,816,344,064.00",HUBS,NaN
6,HUBS,2022-12-19,"13,966,629,000.00",HUBS,NaN
7,HUBS,2022-12-16,"14,353,295,600.00",HUBS,NaN
8,HUBS,2022-12-15,"14,546,628,900.00",HUBS,NaN
9,HUBS,2022-12-14,"15,231,067,500.00",HUBS,NaN


In [39]:
def get_prices(ticker):
    
    
    prices_request = requests.get(f'https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?apikey={api}').json()
    

    #dates_df = pd.DataFrame.from_dict(prices_request,orient='columns')
    out_file = open(f"Prices/{ticker}.json", "w") 
    json.dump(prices_request, out_file, indent = 4) 
    out_file.close() 
    
    with open(f'Prices/{ticker}.json','r') as f:
        data = json.loads(f.read())
    # Flatten data
    df_nested_list = pd.json_normalize(data)
    df_nested_list['Ticker'] = ticker
    df_nested_list.to_csv(f"Prices/{ticker}.csv")
    
    

In [40]:
# call the function
result_price = [get_prices(ticker) for ticker in tickers]

In [42]:
path = '/Users/brendan/Desktop/Python/Software 2020/Prices'
csv_files = [f for f in os.listdir(path) if f.endswith('csv')]
raw_data_df = []
for file in csv_files:
    file = pd.read_csv('/Users/brendan/Desktop/Python/Software 2020/Prices/'+file)
    df = pd.DataFrame(file)
    #df['ticker'] = file
    raw_data_df.append(df)
master_df_prices = pd.concat(raw_data_df, ignore_index=True)
#master_df_price = master_df.drop('Unnamed: 0',axis=1)
master_df_prices

,Unnamed: 0,Error Message,Ticker
0,0,Limit Reach . Please upgrade your plan or visit our documentation for more details at https://financialmodelingprep.com/developer/docs/pricing,CSCO
1,0,Limit Reach . Please upgrade your plan or visit our documentation for more details at https://financialmodelingprep.com/developer/docs/pricing,HUBS
2,0,Limit Reach . Please upgrade your plan or visit our documentation for more details at https://financialmodelingprep.com/developer/docs/pricing,AEIS
3,0,Limit Reach . Please upgrade your plan or visit our documentation for more details at https://financialmodelingprep.com/developer/docs/pricing,FEIM
4,0,Limit Reach . Please upgrade your plan or visit our documentation for more details at https://financialmodelingprep.com/developer/docs/pricing,PRGX
5,0,Limit Reach . Please upgrade your plan or visit our documentation for more details at https://financialmodelingprep.com/developer/docs/pricing,ASYS
6,0,Limit Reach . Please upgrade your plan or visit our documentation for more details at https://financialmodelingprep.com/developer/docs/pricing,ADTN
7,0,Limit Reach . Please upgrade your plan or visit our documentation for more details at https://financialmodelingprep.com/developer/docs/pricing,EVTC
8,0,Limit Reach . Please upgrade your plan or visit our documentation for more details at https://financialmodelingprep.com/developer/docs/pricing,SHSP
9,0,Limit Reach . Please upgrade your plan or visit our documentation for more details at https://financialmodelingprep.com/developer/docs/pricing,FSCT
